In [ ]:
%ntbl pull datasets "hh_ru/hh_ru_dataset.csv"
%ntbl pull datasets "hh_ru/profession_type.xlsx"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
#import statsmodels.stats.tests.test_influence

In [ ]:
df = pd.read_csv('../datasets/hh_ru/hh_ru_dataset.csv', sep=',')
df.info()

In [ ]:
df_cleared = df.copy()
len(df_cleared)
#len(df_cleared[df_cleared.compensation_to.isna()])

In [ ]:
# ожидаемые зарплаты без выбросов
quantile_param = 0.03
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['expected_salary'].quantile(quantile_param),
                       df_prof['expected_salary'].quantile(1 - quantile_param))

df_cleared.loc[:,'exp_salary_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'exp_salary_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]

# проверка по ожидаемой зарплате
df_cleared = df_cleared[(df_cleared['expected_salary'] >= df_cleared['exp_salary_min_limit']) & (df_cleared['expected_salary'] <= df_cleared['exp_salary_max_limit'])]
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
quantile_param = 0.01

quantile_year = (df_cleared['year_of_birth'].quantile(quantile_param),
                       df_cleared['year_of_birth'].quantile(1 - quantile_param))

year_min = quantile_year[0] 
year_min = 1960
df_cleared = df_cleared[(df_cleared['expected_salary'] >= year_min)]
len(df_cleared)

# стаж работы с 16 лет; работал не больше, чем до пенсии
df_cleared = df_cleared[(2023 - df_cleared['work_experience_months'] / 12 - df_cleared['year_of_birth'] >= 16) & (df_cleared['work_experience_months'] <= 63 * 12)]
len(df_cleared)
# print(500000 - len(df_cleared))

In [ ]:
# финальный статус - invitation или discrard
df_cleared = df_cleared[-((df.final_state == 'response') | (df.final_state.isna()))]
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
df_cleared = df_cleared[df.education_level != 'doctor']
len(df_cleared)

In [ ]:
# избавляемся от дубликатов уникального показателя
df_cleared = df_cleared.drop_duplicates(['topic_id'])
len(df_cleared)

In [ ]:
# проверка на логику между compensation_from и compensation_to
df_cleared = df_cleared[(df_cleared.compensation_to.isna()) | (df_cleared.compensation_to == 0.0) | (df_cleared['compensation_from'] <= df_cleared['compensation_to'])]
len(df_cleared)

In [ ]:
# избавляемся от незаполненных полей
# df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state', 'work_schedule', 'compensation_from', 'compensation_to'])
df_cleared = df_cleared.dropna(axis=0, subset=['education_level', 'initial_state', 'work_schedule'])
len(df_cleared)
#print(500000 - len(df_cleared))

In [ ]:
# замена final_state, gender и education_level
fs_dict = {'discard': 0, 'invitation': 1}
gender_dict = {'female': 0, 'male': 1}
# education_dict = {'secondary': 1, 'special_secondary': 2, 'unfinished_higher': 3,
#                'bachelor': 4, 'master': 5, 'higher': 6, 'candidate': 7, 'doctor': 8}
df_cleared = df_cleared.replace({'final_state': fs_dict, 'gender': gender_dict})
df_cleared.info()

In [ ]:
# df_cleared = df_cleared.assign(work_experience_months_norm = df_cleared['work_experience_months'] / 600)
df_cleared = df_cleared.assign(education_level_is_high = ((df_cleared['education_level'] == 'bachelor') | (df_cleared['education_level'] == 'master') | (df_cleared['education_level'] == 'higher') | (df_cleared['education_level'] == 'bachelor')))
df_cleared.education_level_is_high = df_cleared.education_level_is_high.replace({ True : 1 , False : 0 })
df_cleared[['topic_id', 'education_level_is_high']]

In [ ]:
# разделение на группы по профессиям
df_prof = pd.read_excel("../datasets/hh_ru/profession_type.xlsx")

dict_prof = {('Программист, разработчик', 'Бухгалтер', 'Менеджер по логистике, менеджер по ВЭД', 'Оператор ПК, оператор базы данных', 'Менеджер по закупкам', 'Директор магазина, директор сети магазинов', 'Делопроизводитель, архивариус', 'Системный администратор', 'Тестировщик', 'Руководитель проектов', 'Диспетчер', 'Мерчандайзер', 'Аналитик', 'Начальник производства', 'Генеральный директор, исполнительный директор (CEO)', 'Менеджер ресторана', 'Начальник склада', 'Товаровед', 'Менеджер/руководитель АХО', 'Экономист', 'Кладовщик', 'Начальник смены, мастер участка', 'Координатор отдела продаж', 'Технолог', 'Кредитный специалист'): 'symbol', 
            ('Менеджер по продажам, менеджер по работе с клиентами',
'Официант, бармен, бариста', 'Менеджер по персоналу', 'Специалист по подбору персонала', 
'Юрисконсульт', 'Руководитель отдела продаж', 
'Учитель, преподаватель, педагог', 'Офис-менеджер', 'Охранник', 'Воспитатель, няня',
'Психолог', 'Секретарь, помощник руководителя, ассистент', 
'Финансовый менеджер', 'Кассир-операционист', 
'Торговый представитель', 'Врач', 'Администратор', 'Юрист', 
'Продавец-консультант, продавец-кассир', 'Менеджер по туризму', 'Фитнес-тренер, инструктор тренажерного зала', 'Оператор call-центра, специалист контактного центра', 
'Специалист по кадрам', 'Супервайзер', 'Специалист технической поддержки', 'Курьер', 'Уборщица, уборщик'): 'human', 
            ('Инженер-энергетик, инженер-электрик', 'Водитель', 'Сварщик', 
'Инженер-конструктор, инженер-проектировщик', 'Слесарь, сантехник', 
'Упаковщик, комплектовщик', 'Оператор производственной линии', 
'Машинист', 'Механик', 'Сервисный инженер, инженер-механик', 
'Инженер по охране труда и технике безопасности, инженер-эколог', 'Разнорабочий'): 'machinery', 
            ('Дизайнер, художник', 'SMM-менеджер, контент-менеджер', 
'Повар, пекарь, кондитер', 'Менеджер по маркетингу, интернет-маркетолог'): 'art'}


df_cleared.loc[:,'profession_type'] = df_cleared['profession'].apply(lambda x: [value for key, value in dict_prof.items() if x in key][0])
df_cleared[['profession', 'profession_type']]

In [ ]:
len(df[df['profession_type'] == 'machinery'])

In [ ]:
df_cleared.to_excel('clear_final1.xlsx')

In [ ]:
symbol = ['Программист, разработчик', 'Бухгалтер', 'Менеджер по логистике, менеджер по ВЭД', 'Оператор ПК, оператор базы данных', 'Менеджер по закупкам', 'Директор магазина, директор сети магазинов', 'Делопроизводитель, архивариус', 'Системный администратор', 'Тестировщик', 'Руководитель проектов', 'Диспетчер', 'Мерчандайзер', 'Аналитик', 'Начальник производства', 'Генеральный директор, исполнительный директор (CEO)', 'Менеджер ресторана', 'Начальник склада', 'Товаровед', 'Менеджер/руководитель АХО', 'Экономист', 'Кладовщик', 'Начальник смены, мастер участка', 'Координатор отдела продаж', 'Технолог', 'Кредитный специалист']

df_symbol = df_cleared.loc[df_cleared['profession'].isin(symbol)]

#define response variable
y = df_symbol['final_state']

#define explanatory variable
x = df_symbol[['education_level_norm', 'work_experience_months_norm', 'gender']]

#add constant to predictor variables
x = sm.add_constant (x)

#fit linear regression model
model = sm. OLS (y, x). fit ()

#view model summary
print(model. summary ())

In [ ]:
symbol = ['Программист, разработчик', 'Бухгалтер', 'Менеджер по логистике, менеджер по ВЭД', 'Оператор ПК, оператор базы данных', 'Менеджер по закупкам', 'Директор магазина, директор сети магазинов', 'Делопроизводитель, архивариус', 'Системный администратор', 'Тестировщик', 'Руководитель проектов', 'Диспетчер', 'Мерчандайзер', 'Аналитик', 'Начальник производства', 'Генеральный директор, исполнительный директор (CEO)', 'Менеджер ресторана', 'Начальник склада', 'Товаровед', 'Менеджер/руководитель АХО', 'Экономист', 'Кладовщик', 'Начальник смены, мастер участка', 'Координатор отдела продаж', 'Технолог', 'Кредитный специалист']

df_symbol = df_cleared.loc[df_cleared['profession'].isin(symbol)]

# #define response variable
# y = df_symbol['final_state']

# #define explanatory variable
# x = df_symbol[['education_level_norm', 'work_experience_months_norm', 'gender']]

# #add constant to predictor variables
# x = sm.add_constant (x)

# #fit linear regression model
# model = sm. OLS (y, x). fit ()

# #view model summary
# print(model. summary ())





# model_1 = sm.GLM(
#     df_human["final_state"],
#     df_human[['education_level_norm', 'work_experience_months_norm', 'gender']],
#     family=families.Binomial(),
# ).fit()
# print(model_1.summary())


model_1 = sm.GLM(
    df_symbol["final_state"],
    df_symbol[['education_level_is_high', 'gender']],
    family=families.Binomial(),
).fit()
print(model_1.summary())


In [ ]:
human = ['Менеджер по продажам, менеджер по работе с клиентами',
'Официант, бармен, бариста', 'Менеджер по персоналу', 'Специалист по подбору персонала', 
'Юрисконсульт', 'Руководитель отдела продаж', 
'Учитель, преподаватель, педагог', 'Офис-менеджер', 'Охранник', 'Воспитатель, няня',
'Психолог', 'Секретарь, помощник руководителя, ассистент', 
'Финансовый менеджер', 'Кассир-операционист', 
'Торговый представитель', 'Врач', 'Администратор', 'Юрист', 
'Продавец-консультант, продавец-кассир', 'Менеджер по туризму', 'Фитнес-тренер, инструктор тренажерного зала', 'Оператор call-центра, специалист контактного центра', 
'Специалист по кадрам', 'Супервайзер', 'Специалист технической поддержки', 'Курьер', 'Уборщица, уборщик']

df_human = df_cleared.loc[df_cleared['profession'].isin(human)]

# #define response variable
# y = df_human['final_state']

# #define explanatory variable
# x = df_human[['education_level_norm', 'work_experience_months_norm', 'gender']]

# #add constant to predictor variables
# x = sm.add_constant (x)

# #fit linear regression model
# model = sm. OLS (y, x). fit ()

# #view model summary
# print(model. summary ())

model_1 = sm.GLM(
    df_human["final_state"],
    df_human[['education_level_is_high', 'gender']],
    family=families.Binomial(),
).fit()
print(model_1.summary())

In [ ]:
machinery = ['Инженер-энергетик, инженер-электрик', 'Водитель', 'Сварщик', 
'Инженер-конструктор, инженер-проектировщик', 'Слесарь, сантехник', 
'Упаковщик, комплектовщик', 'Оператор производственной линии', 
'Машинист', 'Механик', 'Сервисный инженер, инженер-механик', 
'Инженер по охране труда и технике безопасности, инженер-эколог', 'Разнорабочий']

df_machinery = df_cleared.loc[df_cleared['profession'].isin(machinery)]

# #define response variable
# y = df_machinery['final_state']

# #define explanatory variable
# x = df_machinery[['education_level_norm', 'work_experience_months_norm', 'gender']]

# #add constant to predictor variables
# x = sm.add_constant (x)

# #fit linear regression model
# model = sm. OLS (y, x). fit ()

# #view model summary
# print(model. summary ())

model_1 = sm.GLM(
    df_machinery["final_state"],
    df_machinery[['education_level_norm', 'gender']],
    family=families.Binomial(),
).fit()
print(model_1.summary())

In [ ]:
art = ['Дизайнер, художник', 'SMM-менеджер, контент-менеджер', 
'Повар, пекарь, кондитер', 'Менеджер по маркетингу, интернет-маркетолог']


df_art = df_cleared.loc[df_cleared['profession'].isin(art)]

# #define response variable
# y = df_art['final_state']

# #define explanatory variable
# x = df_art[['education_level_norm', 'work_experience_months_norm', 'gender']]

# #add constant to predictor variables
# x = sm.add_constant (x)

# #fit linear regression model
# model = sm. OLS (y, x). fit ()

# #view model summary
# print(model. summary ())

model_1 = sm.GLM(
    df_art["final_state"],
    df_art[['education_level_is_high', 'gender']],
    family=families.Binomial(),
).fit()
print(model_1.summary())

In [ ]:
df.isnull().sum()